#Import pour data desc

Si on arrive à ces performances on est satisfait :


In [1]:
# prompt: Fait moi l'import des 5 librairies les plus populaires pour le traitement de données

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

# prompt: Peux tu me faire l'import des librairies sklearn pour une regression logisitque et un arbre de décision

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


!pip install ydata_profiling
from ydata_profiling import ProfileReport

DEPRECATION: Loading egg at c:\logiciels\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


   ---------------------------------------- 0.0/300.2 kB ? eta -:--:--
   ------------ --------------------------- 92.2/300.2 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 300.2/300.2 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: wordcloud
    Found existing installation: wordcloud 0.0.0
    Uninstalling wordcloud-0.0.0:
      Successfully uninstalled wordcloud-0.0.0


#Import

In [2]:
df = pd.read_csv('revenus.csv')

In [40]:
profile = ProfileReport(df, title="Profiling Report")
profile.to_file("output.html")

/usr/local/lib/python3.10/dist-packages/ydata_profiling/profile_report.py:363: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

#Premier traitement

In [3]:
df_model=df.replace({">50K":0,"<=50K":1})

C:\Users\192846\AppData\Local\Temp\ipykernel_17596\51222051.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_model=df.replace({">50K":0,"<=50K":1})


https://www.kaggle.com/code/chrisbss1/cramer-s-v-correlation-matrix

In [4]:
from scipy.stats import chi2_contingency
import numpy as np

df_cramer=pd.get_dummies(df)

def cramers_V(var1,var2) :
  crosstab =np.array(pd.crosstab(var1,var2, rownames=None, colnames=None)) # Cross table building
  stat = chi2_contingency(crosstab)[0] # Keeping of the test statistic of the Chi2 test
  obs = np.sum(crosstab) # Number of observations
  mini = min(crosstab.shape)-1 # Take the minimum value between the columns and the rows of the cross table
  return np.sqrt(stat/(obs*mini))

rows= []

for var1 in df_cramer:
  col = []
  for var2 in df_cramer :
    cramers =cramers_V(df_cramer[var1], df_cramer[var2]) # Cramer's V test
    col.append(round(cramers,2)) # Keeping of the rounded value of the Cramer's V
  rows.append(col)

NameError: name 'data_encoded' is not defined

In [10]:
cramers_results = np.array(rows)
df = pd.DataFrame(cramers_results, columns = df_cramer.columns, index =df_cramer.columns)
sns.heatmap(df)

IndexError: Inconsistent shape between the condition and the input (got (110, 1) and (110,))

#Mes premiers modèles

In [37]:
def models_training(df):
  mod_log = LogisticRegression()
  mod_tree = DecisionTreeClassifier()
  X_train,X_test,y_train,y_test = train_test_split(pd.get_dummies(df.drop('income',axis=1)),df.income,test_size=0.2)
  mod_log.fit(X_train,y_train)
  mod_tree.fit(X_train,y_train)
  return mod_log,mod_tree

def models_scoring(modele):
  pass

In [38]:
mod_log,mod_tree=models_training(df_model)

#Traitement des données

##Types

In [15]:
print(df.info())
#On change le type de fnlwgt
df.fnlwgt=df.fnlwgt.astype('float64')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB
None


Plus de soucis de types sur les données

##Frequence

In [16]:
def clean_dataset(df):
  df=df.dropna()
  df=df.reset_index(drop=True)
  for i in df.columns:
    cache=dict(df[i].value_counts())
    seuil=sum(cache.values())*0.05
    for z in range(len(df[i])):
      if cache[df[i][z]]<seuil and type(df[i][0])!="float64":
        df[i][z]="Other"
      if df[i][z]=="?":
        df=df.drop(z)
    df=df.reset_index(drop=True)
#df[i].mode()[0] #A changer ici pour modifier nos traitements (ici le plus fréquent)
  return df
df_other=clean_dataset(df.copy())

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i][z]="Other"
<ipython-input-16-47bcb241f771>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i][z]="Other"
<ipython-input-16-47bcb241f771>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i][z]="Other"
<ipython-input-16-47bcb241f771>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



In [ ]:
df_other

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
